In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.ensemble import GradientBoostingClassifier as XgBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, roc_auc_score,f1_score
from sklearn.metrics import average_precision_score as auc
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.utils import resample as resample
from collections import Counter
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from joblib import dump, load
from imblearn.over_sampling import SMOTE

In [47]:
data = pd.read_csv('final_dataset1.csv')

In [48]:
data = data.drop(['Unnamed: 0'], axis=1)

In [49]:
data = data.dropna()

In [50]:
data

,Age,Height,Weight,Centre-Back,Right Winger,Left Winger,Goalkeeper,Central Midfield,Right-Back,Left-Back,...,head_days,minor_,minor_days,foot,lower_leg,upper_leg,upper_body,arms,head,minor
0,30.0,193,87,1,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,0
1,29.0,175,71,0,1,0,0,0,0,0,...,0.0,3,99.0,0,0,0,0,0,0,1
2,29.0,175,69,0,0,1,0,0,0,0,...,0.0,4,114.0,1,0,1,1,0,0,1
3,29.0,193,91,0,0,0,1,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,1
4,31.0,182,67,0,0,0,0,1,0,0,...,0.0,0,0.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,16.0,195,96,1,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,0
1273,17.0,173,61,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,1
1274,29.0,182,75,0,0,0,0,0,0,1,...,17.0,2,25.0,0,0,1,1,0,0,1
1275,18.0,178,68,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0,1,0,0,1


In [51]:
y_data_foot = data['foot']
y_data_lower_leg = data['lower_leg']
y_data_upper_leg = data['upper_leg']
y_data_upper_body = data['upper_body']
y_data_arms = data['arms']
y_data_head = data['head']
y_data_minor = data['minor']

In [52]:
sum(y_data_foot)/len(data)

0.10806577916992952

In [53]:
sum(y_data_lower_leg)/len(data)

0.08144087705559906

In [54]:
sum(y_data_upper_leg)/len(data)

0.1331245105716523

In [55]:
sum(y_data_upper_body)/len(data)

0.11041503523884104

In [56]:
sum(y_data_arms)/len(data)

0.14095536413469067

In [57]:
sum(y_data_head)/len(data)

0.014095536413469069

In [58]:
y_cols = ['foot', 'lower_leg', 'upper_leg', 'upper_body', 'arms', 'head', 'minor']
x_data = data.drop(y_cols, axis=1)

In [59]:
x_data.shape

(1277, 30)

In [60]:
def resample_data(x,y):
    
    X_oversampled, y_oversampled = resample(x[y == 1],
                                            y[y == 1],
                                            replace=True,
                                            n_samples=int(x[y == 0].shape[0]*.7))

    x = np.vstack((x, X_oversampled))
    y = np.hstack((y, y_oversampled))

    return x,y

In [61]:
random_ind=np.random.choice(len(data),round(len(data)*.4),replace=False)

In [62]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
x_train,y_train=resample_data(x_train,y_train)

In [63]:
len(x_train)

1247

In [64]:
Counter(y_train)

Counter({0: 688, 1: 559})

In [292]:
def foot_NB1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
    #x_train,y_train=resample_data(x_train,y_train)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_foot = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc, recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,'Foot_NB.joblib')

In [296]:
def foot_NB_resam():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
    x_train,y_train=resample_data(x_train,y_train)

    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_foot = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc, recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    
    nb=NB().fit(x_train,y_train)
    
    foot_predictions = nb.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc, recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    

In [84]:
def foot_NB():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
    #x_train,y_train=resample_data(x_train,y_train)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    nb=NB().fit(x_train,y_train)
    
    foot_predictions = nb.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc, recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    

In [85]:
foot_NB()

0.5322896281800391 0.1554770318021201 0.1105373852962771 0.36666666666666664 0.09865470852017937


In [293]:
foot_NB1()

0.6320939334637965 0.13761467889908258 0.11179679952438752 0.25 0.0949367088607595
{'var_smoothing': 3.727593720314938e-07}


In [297]:
foot_NB_resam()

0.5733855185909981 0.2043795620437956 0.12368149922880431 0.4666666666666667 0.1308411214953271
{'var_smoothing': 4.71486636345739e-06}
0.3424657534246575 0.19617224880382778 0.11544084152372207 0.6833333333333333 0.11452513966480447


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [121]:
def foot_XG1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_foot=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,'xg_foot.joblib')

In [298]:
def foot_XG():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    xg = XgBoost().fit(x_train, y_train)
    foot_predictions = xg.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    dump(xg,"Foot_XG.joblib")

In [ ]:
def foot_XG_resam():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=614)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_foot=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,'xg_foot.joblib')

In [269]:
foot_XG()

0.8590998043052838 0.14285714285714288 0.13067514677103717 0.1 0.25


In [124]:
foot_XG1()

0.8551859099804305 0.05128205128205128 0.11720663912444734 0.03333333333333333 0.1111111111111111
{'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': 7, 'learning_rate': 0.2}


In [103]:
def foot_LR1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000)
    params_lr= {
    'C':np.logspace(-3,3,num=50)}
    NB_foot=RandomizedSearchCV(lr,params_lr, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,'lr_foot.joblib')

In [104]:
def foot_LR():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000).fit(x_train,y_train)

    foot_predictions = lr.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [105]:
foot_LR()

0.8669275929549902 0.05555555555555555 0.1058764787836325 0.037037037037037035 0.1111111111111111


In [106]:
foot_LR1()

0.8747553816046967 0.058823529411764705 0.10705225773718924 0.037037037037037035 0.14285714285714285
{'C': 104.81131341546853}


In [125]:
def foot_RF():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_foot, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    NB_foot=RandomForestClassifier().fit(x_train,y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [126]:
foot_RF()

0.8786692759295499 0.06060606060606061 0.10793409195235679 0.037037037037037035 0.16666666666666666


In [111]:
def lower_leg_NB1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=12)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_lower_leg = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    lower_leg_predictions = NB_lower_leg.predict(x_test)
    lower_leg_score = accuracy_score(y_test, lower_leg_predictions)
    lower_leg_f1_score = f1_score(y_test, lower_leg_predictions)
    lower_leg_auc = auc(y_test, lower_leg_predictions)
    print(lower_leg_score, lower_leg_f1_score, lower_leg_auc,recall(y_test,lower_leg_predictions),precision(y_test,lower_leg_predictions))
    print(NB_lower_leg.best_params_)


In [118]:
def lower_leg_NB():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=12)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB().fit(x_train, y_train)
    lower_leg_predictions = nb.predict(x_test)
    lower_leg_score = accuracy_score(y_test, lower_leg_predictions)
    lower_leg_f1_score = f1_score(y_test, lower_leg_predictions)
    lower_leg_auc = auc(y_test, lower_leg_predictions)
    print(lower_leg_score, lower_leg_f1_score, lower_leg_auc,recall(y_test,lower_leg_predictions),precision(y_test,lower_leg_predictions))



In [119]:
lower_leg_NB()

0.22700587084148727 0.1280353200883002 0.07416004431302901 0.6904761904761905 0.0705596107055961


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [120]:
lower_leg_NB1()

0.8434442270058709 0.13043478260869565 0.08759295499021526 0.14285714285714285 0.12
{'var_smoothing': 0.4291934260128778}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [290]:
def lower_leg_XG1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=12)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_lower_leg=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    lower_leg_predictions = NB_lower_leg.predict(x_test)
    lower_leg_score = accuracy_score(y_test, lower_leg_predictions)
    lower_leg_f1_score = f1_score(y_test, lower_leg_predictions)
    lower_leg_auc = auc(y_test, lower_leg_predictions)
    print(lower_leg_score, lower_leg_f1_score, lower_leg_auc,recall(y_test,lower_leg_predictions),precision(y_test,lower_leg_predictions))
    print(NB_lower_leg.best_params_)
    dump(NB_lower_leg,'Lleg_XG.joblib')

In [139]:
def lower_leg_XG():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=12)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost().fit(x_train, y_train)
    lower_leg_predictions = xg.predict(x_test)
    lower_leg_score = accuracy_score(y_test, lower_leg_predictions)
    lower_leg_f1_score = f1_score(y_test, lower_leg_predictions)
    lower_leg_auc = auc(y_test, lower_leg_predictions)
    print(lower_leg_score, lower_leg_f1_score, lower_leg_auc,recall(y_test,lower_leg_predictions),precision(y_test,lower_leg_predictions))


In [140]:
lower_leg_XG()

0.8101761252446184 0.1415929203539823 0.0879983094916531 0.19047619047619047 0.11267605633802817


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [291]:
lower_leg_XG1()

0.9099804305283757 0.0 0.0821917808219178 0.0 0.0
{'n_estimators': 150, 'max_features': 'log2', 'max_depth': 9, 'learning_rate': 0.3}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [143]:
def lower_leg_LR1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000)
    params_lr= {
    'C':np.logspace(-3,3,num=50)}
    NB_foot=RandomizedSearchCV(lr,params_lr, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,'lr_foot.joblib')

In [144]:
def lower_leg_LR():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000).fit(x_train, y_train)
    foot_predictions = lr.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))



In [145]:
lower_leg_LR()

0.8845401174168297 0.06349206349206349 0.08281303388935482 0.047619047619047616 0.09523809523809523


In [146]:
lower_leg_LR1()

0.8884540117416829 0.033898305084745756 0.08163539388358082 0.023809523809523808 0.058823529411764705
{'C': 184.20699693267164}


In [288]:
def lower_leg_RF():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_lower_leg, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    NB_foot=RandomForestClassifier().fit(x_train,y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    dump(NB_foot,"Lleg_RF.joblib")

In [289]:
lower_leg_RF()

0.8962818003913894 0.10169491525423728 0.08892598135144468 0.07142857142857142 0.17647058823529413


In [274]:
def upper_leg_NB1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_upper_leg = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    upper_leg_predictions = NB_upper_leg.predict(x_test)
    upper_leg_score = accuracy_score(y_test, upper_leg_predictions)
    upper_leg_f1_score = f1_score(y_test, upper_leg_predictions)
    upper_leg_auc = auc(y_test, upper_leg_predictions)
    print(upper_leg_score, upper_leg_f1_score, upper_leg_auc,recall(y_test,upper_leg_predictions),precision(y_test,upper_leg_predictions))
    print(NB_upper_leg.best_params_)
    dump(NB_upper_leg,"Uleg_NB.joblib")

In [160]:
def upper_leg_NB():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB().fit(x_train, y_train)
    upper_leg_predictions = nb.predict(x_test)
    upper_leg_score = accuracy_score(y_test, upper_leg_predictions)
    upper_leg_f1_score = f1_score(y_test, upper_leg_predictions)
    upper_leg_auc = auc(y_test, upper_leg_predictions)
    print(upper_leg_score, upper_leg_f1_score, upper_leg_auc,recall(y_test,upper_leg_predictions),precision(y_test,upper_leg_predictions))


In [275]:
upper_leg_NB1()

0.6105675146771037 0.2656826568265683 0.15449555456237715 0.5454545454545454 0.17560975609756097
{'var_smoothing': 5.4286754393238595e-09}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [162]:
upper_leg_NB()

0.589041095890411 0.27586206896551724 0.1591057344482002 0.6060606060606061 0.17857142857142858


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [164]:
def upper_leg_XG1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_upper_leg=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    upper_leg_predictions = NB_upper_leg.predict(x_test)
    upper_leg_score = accuracy_score(y_test, upper_leg_predictions)
    upper_leg_f1_score = f1_score(y_test, upper_leg_predictions)
    upper_leg_auc = auc(y_test, upper_leg_predictions)
    print(upper_leg_score, upper_leg_f1_score, upper_leg_auc,recall(y_test,upper_leg_predictions),precision(y_test,upper_leg_predictions))
    print(NB_upper_leg.best_params_)
    dump(NB_upper_leg,'xg_uleg.joblib')

In [165]:
def upper_leg_XG():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost().fit(x_train, y_train)
    upper_leg_predictions = xg.predict(x_test)
    upper_leg_score = accuracy_score(y_test, upper_leg_predictions)
    upper_leg_f1_score = f1_score(y_test, upper_leg_predictions)
    upper_leg_auc = auc(y_test, upper_leg_predictions)
    print(upper_leg_score, upper_leg_f1_score, upper_leg_auc,recall(y_test,upper_leg_predictions),precision(y_test,upper_leg_predictions))

In [166]:
upper_leg_XG()

0.7671232876712328 0.2515723270440252 0.15518737657491524 0.30303030303030304 0.21505376344086022


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [169]:
upper_leg_XG1()

0.8669275929549902 0.028571428571428574 0.13098944434560872 0.015151515151515152 0.25
{'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 8, 'learning_rate': 0.3}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [173]:
def upper_leg_LR1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000)
    params_lr= {
    'C':np.logspace(-3,3,num=50)}
    NB_foot=RandomizedSearchCV(lr,params_lr, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,'lr_foot.joblib')

In [174]:
def upper_leg_LR():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000).fit(x_train, y_train)
    foot_predictions = lr.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [175]:
upper_leg_LR()

0.8180039138943248 0.1308411214953271 0.14441519742503728 0.09859154929577464 0.19444444444444445


In [177]:
upper_leg_LR1()

0.8277886497064579 0.12000000000000001 0.14468578119638947 0.08450704225352113 0.20689655172413793
{'C': 244.205309454865}


In [276]:
def upper_leg_RF():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_leg, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    NB_foot=RandomForestClassifier().fit(x_train,y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    dump(NB_foot,"Uleg_RF.joblib")

In [277]:
upper_leg_RF()

0.8277886497064579 0.18518518518518517 0.1574400121573573 0.14084507042253522 0.2702702702702703


In [278]:
def upper_body_NB1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_upper_body = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    upper_body_predictions = NB_upper_body.predict(x_test)
    upper_body_score = accuracy_score(y_test, upper_body_predictions)
    upper_body_f1_score = f1_score(y_test, upper_body_predictions)
    upper_body_auc = auc(y_test, upper_body_predictions)
    print(upper_body_score, upper_body_f1_score, upper_body_auc,recall(y_test,upper_body_predictions),precision(y_test,upper_body_predictions))
    print(NB_upper_body.best_params_)
    dump(NB_upper_body,'Ubody_NB.joblib')

In [186]:
def upper_body_NB():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB().fit(x_train, y_train)
    upper_body_predictions = nb.predict(x_test)
    upper_body_score = accuracy_score(y_test, upper_body_predictions)
    upper_body_f1_score = f1_score(y_test, upper_body_predictions)
    upper_body_auc = auc(y_test, upper_body_predictions)
    print(upper_body_score, upper_body_f1_score, upper_body_auc,recall(y_test,upper_body_predictions),precision(y_test,upper_body_predictions))


In [187]:
upper_body_NB()

0.37377690802348335 0.23444976076555024 0.13533054924430882 0.765625 0.1384180790960452


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [279]:
upper_body_NB1()

0.5812133072407045 0.21897810218978103 0.1335004892367906 0.46875 0.14285714285714285
{'var_smoothing': 0.00013894954943731373}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [193]:
def upper_body_XG1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_upper_body=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    upper_body_predictions = NB_upper_body.predict(x_test)
    upper_body_score = accuracy_score(y_test, upper_body_predictions)
    upper_body_f1_score = f1_score(y_test, upper_body_predictions)
    upper_body_auc = auc(y_test, upper_body_predictions)
    print(upper_body_score, upper_body_f1_score, upper_body_auc,recall(y_test,upper_body_predictions),precision(y_test,upper_body_predictions))
    print(NB_upper_body.best_params_)

In [280]:
def upper_body_XG():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost().fit(x_train, y_train)
    upper_body_predictions = xg.predict(x_test)
    upper_body_score = accuracy_score(y_test, upper_body_predictions)
    upper_body_f1_score = f1_score(y_test, upper_body_predictions)
    upper_body_auc = auc(y_test, upper_body_predictions)
    print(upper_body_score, upper_body_f1_score, upper_body_auc,recall(y_test,upper_body_predictions),precision(y_test,upper_body_predictions))
    dump(xg,"Ubody_XG.joblib")

In [281]:
upper_body_XG()

0.7729941291585127 0.15942028985507245 0.12926718609509705 0.171875 0.14864864864864866


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [196]:
upper_body_XG1()

0.8688845401174168 0.028985507246376812 0.1264126712328767 0.015625 0.2
{'n_estimators': 50, 'max_features': 'log2', 'max_depth': 9, 'learning_rate': 0.3}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [201]:
def upper_body_LR():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000).fit(x_train, y_train)
    foot_predictions = lr.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [202]:
def upper_body_LR1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000)
    params_lr= {
    'C':np.logspace(-3,3,num=50)}
    NB_foot=RandomizedSearchCV(lr,params_lr, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
  

In [203]:
def upper_body_RF():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_upper_body, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    NB_foot=RandomForestClassifier().fit(x_train,y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [205]:
upper_body_RF()

0.863013698630137 0.07894736842105263 0.10099228732588926 0.058823529411764705 0.12


In [206]:
upper_body_LR()

0.8551859099804305 0.0975609756097561 0.10209669373715018 0.0784313725490196 0.12903225806451613


In [207]:
upper_body_LR1()

0.8590998043052838 0.1 0.10279463698563979 0.0784313725490196 0.13793103448275862
{'C': 138.9495494373136}


In [211]:
def arms_NB():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=1)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB().fit(x_train, y_train)
    arms_predictions = nb.predict(x_test)
    arms_score = accuracy_score(y_test, arms_predictions)
    arms_f1_score = f1_score(y_test, arms_predictions)
    arms_auc = auc(y_test, arms_predictions)
    print(arms_score, arms_f1_score, arms_auc,recall(y_test,arms_predictions),precision(y_test,arms_predictions))


In [212]:
def arms_NB1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=1)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_arms = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    arms_predictions = NB_arms.predict(x_test)
    arms_score = accuracy_score(y_test, arms_predictions)
    arms_f1_score = f1_score(y_test, arms_predictions)
    arms_auc = auc(y_test, arms_predictions)
    print(arms_score, arms_f1_score, arms_auc,recall(y_test,arms_predictions),precision(y_test,arms_predictions))
    print(NB_arms.best_params_)
    dump(NB_arms,'nb_arms.joblib')

In [213]:
arms_NB()

0.324853228962818 0.24836601307189543 0.14661387303164575 0.7808219178082192 0.14766839378238342


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [214]:
arms_NB1()

0.6086105675146771 0.23076923076923078 0.1500779639377544 0.410958904109589 0.16042780748663102
{'var_smoothing': 2.4420530945486497e-07}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [216]:
def arms_XG1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=1)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_arms=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    arms_predictions = NB_arms.predict(x_test)
    arms_score = accuracy_score(y_test, arms_predictions)
    arms_f1_score = f1_score(y_test, arms_predictions)
    arms_auc = auc(y_test, arms_predictions)
    print(arms_score, arms_f1_score, arms_auc,recall(y_test,arms_predictions),precision(y_test,arms_predictions))
    print(NB_arms.best_params_)

In [217]:
def arms_XG():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=1)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost().fit(x_train, y_train)
    arms_predictions = xg.predict(x_test)
    arms_score = accuracy_score(y_test, arms_predictions)
    arms_f1_score = f1_score(y_test, arms_predictions)
    arms_auc = auc(y_test, arms_predictions)
    print(arms_score, arms_f1_score, arms_auc,recall(y_test,arms_predictions),precision(y_test,arms_predictions))

In [218]:
arms_XG()

0.7299412915851272 0.15853658536585366 0.14285714285714285 0.1780821917808219 0.14285714285714285


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [224]:
arms_XG1()

0.8258317025440313 0.0 0.14285714285714285 0.0 0.0
{'n_estimators': 150, 'max_features': 'log2', 'max_depth': 7, 'learning_rate': 0.2}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [282]:
def arms_LR1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000)
    params_lr= {
    'C':np.logspace(-3,3,num=50)}
    NB_foot=RandomizedSearchCV(lr,params_lr, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
    dump(NB_foot,"Arm_LR.joblib")

In [232]:
def arms_LR():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000).fit(x_train, y_train)
    foot_predictions = lr.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [284]:
def arms_RF():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_arms, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    NB_foot=RandomForestClassifier().fit(x_train,y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    dump(NB_foot,"Arm_RF.joblib")

In [234]:
arms_LR()

0.8160469667318982 0.06 0.1606963868073123 0.036585365853658534 0.16666666666666666


In [283]:
arms_LR1()

0.821917808219178 0.09900990099009901 0.16673114460055918 0.06097560975609756 0.2631578947368421
{'C': 323.745754281764}


In [285]:
arms_RF()

0.8160469667318982 0.11320754716981131 0.16702066727125195 0.07317073170731707 0.25


In [286]:
def head_NB():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB().fit(x_train, y_train)
    head_predictions = nb.predict(x_test)
    head_score = accuracy_score(y_test, head_predictions)
    head_f1_score = f1_score(y_test, head_predictions)
    head_auc = auc(y_test, head_predictions)
    print(head_score, head_f1_score, head_auc,recall(y_test,head_predictions),precision(y_test,head_predictions))
    dump(nb,"Head_NB.joblib")

In [241]:
def head_NB1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    nb=NB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=50)}
    NB_head = GridSearchCV(estimator=nb, 
                 param_grid=params_NB, 
                 cv=5,   
                  n_jobs=-1).fit(x_train, y_train)
    head_predictions = NB_head.predict(x_test)
    head_score = accuracy_score(y_test, head_predictions)
    head_f1_score = f1_score(y_test, head_predictions)
    head_auc = auc(y_test, head_predictions)
    print(head_score, head_f1_score, head_auc,recall(y_test,head_predictions),precision(y_test,head_predictions))
    print(NB_head.best_params_)
    dump(NB_head,'nb_head.joblib')

In [242]:
head_NB1()

0.6477495107632094 0.04255319148936171 0.019716455241059205 0.4444444444444444 0.0223463687150838
{'var_smoothing': 4.71486636345739e-06}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [287]:
head_NB()

0.49706457925636005 0.0446096654275093 0.02125545687189523 0.6666666666666666 0.023076923076923078


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [247]:
def head_XG():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost().fit(x_train, y_train)
    head_predictions = xg.predict(x_test)
    head_score = accuracy_score(y_test, head_predictions)
    head_f1_score = f1_score(y_test, head_predictions)
    head_auc = auc(y_test, head_predictions)
    print(head_score, head_f1_score, head_auc,recall(y_test,head_predictions),precision(y_test,head_predictions))


In [248]:
def head_XG1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=500)
    x_train,y_train=resample_data(x_train,y_train)
    xg = XgBoost()
    params_XG= {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'max_features': ['auto','sqrt','log2']}
    NB_head=RandomizedSearchCV(xg,params_XG, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    head_predictions = NB_head.predict(x_test)
    head_score = accuracy_score(y_test, head_predictions)
    head_f1_score = f1_score(y_test, head_predictions)
    head_auc = auc(y_test, head_predictions)
    print(head_score, head_f1_score, head_auc,recall(y_test,head_predictions),precision(y_test,head_predictions))
    print(NB_head.best_params_)

In [249]:
head_XG()

0.9726027397260274 0.0 0.01761252446183953 0.0 0.0


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [250]:
head_XG1()

0.9823874755381604 0.0 0.01761252446183953 0.0 0.0
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.01}


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [254]:
def head_LR1():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000)
    params_lr= {
    'C':np.logspace(-3,3,num=50)}
    NB_foot=RandomizedSearchCV(lr,params_lr, 
                 cv=5,   
                 n_jobs=-1).fit(x_train, y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
    print(NB_foot.best_params_)
  

In [258]:
def head_LR():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    lr=LogisticRegression(max_iter=10000).fit(x_train, y_train)
    foot_predictions = lr.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))
  

In [263]:
def head_RF():
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data_head, test_size=0.4, random_state=0)
    smt = SMOTE(random_state=0)
    x_train, y_train = smt.fit_resample(x_train, y_train)
    NB_foot=RandomForestClassifier().fit(x_train,y_train)
    foot_predictions = NB_foot.predict(x_test)
    foot_score = accuracy_score(y_test, foot_predictions)
    foot_f1_score = f1_score(y_test, foot_predictions)
    foot_auc = auc(y_test, foot_predictions)
    print(foot_score, foot_f1_score, foot_auc,recall(y_test,foot_predictions),precision(y_test,foot_predictions))


In [260]:
head_LR()

0.9647749510763209 0.0 0.019569471624266144 0.0 0.0


In [262]:
head_LR1()

0.974559686888454 0.0 0.019569471624266144 0.0 0.0
{'C': 33.9322177189533}


In [264]:
head_RF()

0.9804305283757339 0.0 0.019569471624266144 0.0 0.0


C:\Users\ksi01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
